In [1]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import tensorflow as tf

/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/linbinyang/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
mnist = read_data_sets("./dataset", one_hot=True, reshape=False, validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Extracting ./dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
CKPT_DIR = 'new_ckpt_dir_3'

In [5]:
class Network3(object):
    def __init__(self):
        self.global_step = tf.Variable(0,trainable = False)
        self.x = tf.placeholder(tf.float32, [None, 28, 28, 1], name = "x_input")
        self.label = tf.placeholder(tf.float32, [None, 10], name = "y_input")
        self.lr = 0.003
        self.W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1))   # 5x5 patch, 1 input channel, K output channels
        self.B1 = tf.Variable(tf.ones([4])/10)
        self.W2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1))
        self.B2 = tf.Variable(tf.ones([8])/10)
        self.W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=0.1))
        self.B3 = tf.Variable(tf.ones([12])/10)
        self.W4 = tf.Variable(tf.truncated_normal([7 * 7 * 12, 200], stddev=0.1))
        self.B4 = tf.Variable(tf.ones([200])/10)
        self.W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
        self.B5 = tf.Variable(tf.ones([10])/10)
        stride = 1  # output is 28x28
        self.Y1 = tf.nn.relu(tf.nn.conv2d(self.x, self.W1, strides=[1, stride, stride, 1], padding='SAME') + self.B1)
        stride = 2  # output is 14x14
        self.Y2 = tf.nn.relu(tf.nn.conv2d(self.Y1, self.W2, strides=[1, stride, stride, 1], padding='SAME') + self.B2)
        stride = 2  # output is 7x7
        self.Y3 = tf.nn.relu(tf.nn.conv2d(self.Y2, self.W3, strides=[1, stride, stride, 1], padding='SAME') + self.B3)
        # reshape the output from the third convolution for the fully connected layer
        self.YY = tf.reshape(self.Y3, shape=[-1, 7 * 7 * 12])
        self.Y4 = tf.nn.relu(tf.matmul(self.YY, self.W4) + self.B4)
        self.Ylogits = tf.matmul(self.Y4, self.W5) + self.B5
        self.predict = tf.nn.softmax(self.Ylogits, name = "z_output")
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = self.Ylogits, labels = self.label)
        self.loss = tf.reduce_mean(self.cross_entropy)*100
        # accuracy of the trained model, between 0 (worst) and 1 (best)
        self.predict = tf.equal(tf.argmax(self.predict, 1), tf.argmax(self.label, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.predict, tf.float32))
        self.train = tf.train.AdamOptimizer(self.lr).minimize(self.cross_entropy, global_step = self.global_step)

In [6]:
class Train(object):
    
    def __init__(self):
        self.net = Network3()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.CKPT_DIR = CKPT_DIR
        
    def train(self):
        batch_size = 64
        train_step = 10000
        step = 0
        #Save Model every 1000 steps
        save_interval = 10000
        saver = tf.train.Saver(max_to_keep = 0)
        ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
        if ckpt and ckpt.get_checkpoint_state(CKPT_DIR):
            saver.restore(self.sess,ckpt.model_checkpoint_path)
            step = self.sess.run(self.net.global_step)
            print('continue from')
            print('  -> Minibatch update : ',step)
        while step < train_step:
            x, label = mnist.train.next_batch(batch_size)
            _,loss = self.sess.run([self.net.train,self.net.loss], feed_dict = {self.net.x: x, self.net.label:label})
            step = self.sess.run(self.net.global_step)
            if step % 1000 == 0:
                print('No %6d step, current loss: %.3f'%(step,loss))
            if step % save_interval == 0:
                saver.save(self.sess,CKPT_DIR + '/model',global_step = step)
    
    def calculate_accuracy(self):
        test_x = mnist.train.images
        test_label = mnist.train.labels
        acc = self.sess.run(self.net.accuracy,feed_dict = {self.net.x:test_x,self.net.label:test_label})
        print("acc: %.3f, Number of pic test: %d " % (acc, len(test_label)))

In [7]:
if __name__ == '__main__':
    model = Train()
    model.train()
    model.calculate_accuracy()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

No   1000 step, current loss: 3.665
No   2000 step, current loss: 10.648
No   3000 step, current loss: 4.986
No   4000 step, current loss: 7.531
No   5000 step, current loss: 0.071
No   6000 step, current loss: 1.419
No   7000 step, current loss: 0.136
No   8000 step, current loss: 0.007
No   9000 step, current loss: 0.089
No  10000 step, current loss: 0.070
INFO:tensorflow:new_ckpt_dir_3/model-10000 is not in all_model_checkpoint_paths. Manually adding it.
acc: 0.997, Number of pic test: 60000 


In [8]:
import numpy as np
y, label = mnist.train.next_batch(32)

In [9]:
def model_restore_inference():
    sess = tf.Session()
    saver = tf.train.import_meta_graph('./new_ckpt_dir_3/model-10000.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./new_ckpt_dir_3'))
    input_x = sess.graph.get_tensor_by_name('x_input:0')
    logits = sess.graph.get_tensor_by_name('z_output:0')
    ret = sess.run(logits, {input_x:y})
    print (ret.tolist())
    final_res = np.argmax(ret.tolist(), 1)
    print (final_res)

In [10]:
model_restore_inference()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./new_ckpt_dir_3/model-10000
[[1.0, 8.685638769876874e-22, 5.1947792042376234e-14, 4.9062332061786216e-17, 2.5662080950494686e-21, 7.945626838015597e-20, 2.4465341777072996e-15, 7.559778583365287e-16, 4.23344695345447e-14, 2.34717734137746e-09], [2.501010042918776e-15, 4.381198868994407e-21, 1.3664994436859215e-22, 1.2814010025377234e-25, 4.766344391450594e-17, 3.2966294005288305e-17, 1.0, 1.608597857087253e-26, 1.128747370483812e-16, 2.0269140010168406e-26], [3.8362623624652037e-13, 9.839298192664773e-15, 0.9999350309371948, 9.59910262388064e-10, 1.8024191432147588e-16, 1.7118948792571196e-16, 1.5607156976773433e-15, 1.0079787005068397e-09, 6.499326991615817e-05, 1.2921555373000132e-14], [2.0735156134716954e-20, 1.8053586122395273e-15, 1.7024084914126977e-14, 1.0, 2.956377919978003e-16, 1.1622850459858114e-09, 1.2682233889989514e-16, 4.071856445227694e-14, 1

In [11]:
unit = {}
unit['first'] = y.tolist()
import json
with open ("data15.json", "w") as fp:
    fp.write(json.dumps(unit, indent=4))